In [8]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar

#Logger
logging.basicConfig(filename='Multigraph.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
#########################
# PREPARE MATRIX 1 DATA #
#########################


COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]
PARTY_NAMES_LIST = ['vox_es', 'populares', 'CiudadanosCs', 'socialistes_cat',
                    'EnComu_Podem','Esquerra_ERC', 'cupnacional', 'JuntsXCat']


def load_ideologial_positions(file_name):
    with open(file_name, 'r') as fp:
        positions = json.load(fp)       
    return positions

positions = load_ideologial_positions('../Cat-ideological_position_dict.json')
print(positions)

matrix1 = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(positions[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(A)]])
        b = np.array(positions[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(B)]])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix1[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance
        
print(matrix1)

In [ ]:
#########################
# PREPARE MATRIX 2 DATA #
#########################

########################
#          CEO         #
########################

CEO_ideological_position ={}

CEO_ideological_position['CiudadanosCs'] = [5.50, 4.38]
CEO_ideological_position['JuntsXCat'] = [4.15, 8.77]
CEO_ideological_position['Esquerra_ERC'] = [3.37, 7.71]
CEO_ideological_position['socialistes_cat'] = [4.08, 4.75]
CEO_ideological_position['EnComu_Podem'] = [2.95, 4.80]
CEO_ideological_position['cupnacional'] = [2.11, 8.02]
CEO_ideological_position['populares'] = [6.42, 4.56]
CEO_ideological_position['vox_es'] = [6.84, 4.00]

print(CEO_ideological_position)

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]
PARTY_NAMES_LIST = ['vox_es', 'populares', 'CiudadanosCs', 'socialistes_cat',
                    'EnComu_Podem','Esquerra_ERC', 'cupnacional', 'JuntsXCat']


matrix2 = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(CEO_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(A)]])
        b = np.array(CEO_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(B)]])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix2[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        
print(matrix2)  

In [ ]:
########################
# PREPARE MATRIX 3 DATA #
########################

#############################################
#          MANIFESTO RESEARCH GROUP         #
#############################################

Manifesto_ideological_position ={}

Manifesto_ideological_position['CS'] = [6.04, 4.39]
Manifesto_ideological_position['JXCAT'] = [4.94, 6.29]
Manifesto_ideological_position['ERC'] = [2.94, 7.11]
Manifesto_ideological_position['PSC'] = [5.31, 5.36]
Manifesto_ideological_position['COMUNS'] = [2.70, 5.78]
Manifesto_ideological_position['CUP'] = [1.51, 6.84]
Manifesto_ideological_position['PP'] = [9.57, 4.67]
Manifesto_ideological_position['VOX'] = [8.33, 1.32]

print(Manifesto_ideological_position)

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]


matrix3 = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(Manifesto_ideological_position[A])
        b = np.array(Manifesto_ideological_position[B])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix3[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        
print(matrix3)  

In [ ]:
########################
# PREPARE MATRIX 4 DATA #
########################

########################
#          CHES         #
########################

CHES_ideological_position ={}

CHES_ideological_position['PODEMOS'] = [1.93]
CHES_ideological_position['IU'] = [1.86]
CHES_ideological_position['MAS_PAIS'] = [2.73]
CHES_ideological_position['PSOE'] = [3.60]
CHES_ideological_position['CS'] = [7.20]
CHES_ideological_position['PP'] = [8.06]
CHES_ideological_position['VOX'] = [9.71]
CHES_ideological_position['ERC'] = [3.20]
CHES_ideological_position['PDECAT'] = [6.66]

print(CHES_ideological_position)

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSOE", "PODEMOS", "ERC", "CUP", "PDECAT"]


matrix4 = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        if(A != "CUP" and B != "CUP"):
            a = np.array(CHES_ideological_position[A])
            b = np.array(CHES_ideological_position[B])
            distance = np.linalg.norm(a-b)
            #https://en.wikipedia.org/wiki/Euclidean_distance

            matrix4[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        else:
            matrix4[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = 0
        
print(matrix4)  

In [ ]:
################################
# PLOT HEATMAP (NO REDUNDANCY) #
################################

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]
matrix_df1 = pd.DataFrame(data=matrix1, index=COMMUNITIES_LIST, columns=COMMUNITIES_LIST)
matrix_df2 = pd.DataFrame(data=matrix2, index=COMMUNITIES_LIST, columns=COMMUNITIES_LIST)

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]
matrix_df3 = pd.DataFrame(data=matrix3, index=COMMUNITIES_LIST, columns=COMMUNITIES_LIST)

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]
matrix_df4 = pd.DataFrame(data=matrix4, index=COMMUNITIES_LIST, columns=COMMUNITIES_LIST)

# Calculate pairwise-correlation
m1 = matrix_df1.corr()
m2 = matrix_df2.corr()
m3 = matrix_df3.corr()
m4 = matrix_df4.corr()

# Create a mask
mask1 = np.triu(np.ones_like(m1, dtype=bool))
mask2 = np.triu(np.ones_like(m2, dtype=bool))
mask3 = np.triu(np.ones_like(m3, dtype=bool))
mask4 = np.triu(np.ones_like(m4, dtype=bool))

np.fill_diagonal(mask1, False)
np.fill_diagonal(mask2, False)
np.fill_diagonal(mask3, False)
np.fill_diagonal(mask4, False)

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

fig, axn = plt.subplots(2, 2, figsize = (10,12))
#fig, axn = plt.subplots(2, 2, figsize = (10,10), sharex=True, sharey=True)
cbar_ax = fig.add_axes([.91, .3, .03, .4])

#Normalize axes
import matplotlib.colors as mcolors
values = np.hstack([d.values.ravel() for d in [matrix_df1, matrix_df2, matrix_df3, matrix_df4]])
norm = mcolors.Normalize(values.min(), values.max()) 

sns.heatmap(matrix_df1, mask=mask1, center=0, annot=True, fmt='.2f', square=True, cmap=cmap, norm=norm, ax=axn[0, 0], cbar=False)
sns.heatmap(matrix_df2, mask=mask2, center=0, annot=True, fmt='.2f', square=True, cmap=cmap, norm=norm, ax=axn[0, 1], cbar=False)
sns.heatmap(matrix_df3, mask=mask3, center=0, annot=True, fmt='.2f', square=True, cmap=cmap, norm=norm, ax=axn[1, 0], cbar=False)
sns.heatmap(matrix_df4, mask=mask3, center=0, annot=True, fmt='.2f', square=True, cmap=cmap, norm=norm, ax=axn[1, 1], cbar_ax=cbar_ax)

axn[0, 0].set_title("TIDEM")
axn[0, 1].set_title("CEO")
axn[1, 0].set_title("Manifesto")
axn[1, 1].set_title("CHES")

#plt.title("Ideological Distance (Method vs CES vs MANIFESTO vs CHES)")

#plt.savefig("CAT-MULTIPGRAPH.png")
plt.show()

In [ ]:
matrix_df1

In [ ]:
matrix_df1.corrwith(matrix_df2)

In [ ]:
matrix_df1.corrwith(matrix_df3)

In [ ]:
matrix_df1.corrwith(matrix_df4)